In [1]:
from architecture import ContextMatcher
import json
import torch
import torch.nn.functional as F
import numpy as np
import nltk
from tqdm.auto import tqdm
data_dir = '/hdd/unsupervised-summarization/'

In [2]:
vocab = json.load(open(data_dir+'data-giga/vocab.json', 'r'))

In [5]:
embeddings = np.load(data_dir+'data-giga/embeddings.npy')
embeddings = torch.from_numpy(embeddings).cuda()
elmo = torch.load(data_dir+'elmo_model')
elmo_uni = torch.load(data_dir+'elmo_model_uni')
# lm_elmo = torch.load(data_dir+'LM-check')
with open(data_dir+'data-giga/lm_record.json') as f:
    lm_record = json.load(f)

In [6]:
from StatisticalLM import StatisticalLM
lm_statis = StatisticalLM(lm_record, vocab)

In [8]:
matcher = ContextMatcher(embeddings, elmo, lm_statis)
matcher.eval()
matcher_uni = ContextMatcher(embeddings, elmo_uni, lm_statis)
matcher.eval()

ContextMatcher(
  (pretrained_elmo): Elmo(
    (_elmo_lstm): _ElmoBiLm(
      (_token_embedder): _ElmoCharacterEncoder(
        (char_conv_0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
        (char_conv_1): Conv1d(16, 32, kernel_size=(2,), stride=(1,))
        (char_conv_2): Conv1d(16, 64, kernel_size=(3,), stride=(1,))
        (char_conv_3): Conv1d(16, 128, kernel_size=(4,), stride=(1,))
        (char_conv_4): Conv1d(16, 256, kernel_size=(5,), stride=(1,))
        (char_conv_5): Conv1d(16, 512, kernel_size=(6,), stride=(1,))
        (char_conv_6): Conv1d(16, 1024, kernel_size=(7,), stride=(1,))
        (_highways): Highway(
          (_layers): ModuleList(
            (0): Linear(in_features=2048, out_features=4096, bias=True)
            (1): Linear(in_features=2048, out_features=4096, bias=True)
          )
        )
        (_projection): Linear(in_features=2048, out_features=512, bias=True)
      )
      (_word_embedding): Embedding()
      (_elmo_lstm): ElmoLstm(
        (fo

In [35]:
raw1 = "turnout was heavy for parliamentary elections monday in trinidad and tobage after a month of intensive campaigning throughout the country, one of the most prosperous in the caribbean."
raw2 = "A dozen current and former staff from different areas of the State Department and at different stages of their careers who spoke to CNN said the Ukraine controversy has exacerbated divisions between political appointees and career diplomats, many of whom say the department is being politicized in ways that undermine US ties to other countries."

In [38]:
s = [list(nltk.word_tokenize(raw)) for raw in [raw1, raw2]]

In [13]:
vocab_inv = {a:b for b,a in vocab.items()}

In [14]:
def similarity(x, y):
    return F.cosine_similarity(x, y, -1)

In [119]:
def count_score(source, now, target):
    max_score = -1
    max_index = 0

    for i in range(now+1, len(source)):
        score = similarity(source[i], target)
        if max_score < 0 or score > max_score:
            max_score = score
            max_index = i
            
    return max_index, max_score

In [150]:
def summary(sourse, cand, matcher):
    output = []
    now = -1
    matcher.voronoi_split(torch.LongTensor(cand))
    
    while now < len(sourse)-1:

        index_list = []
        score_list = []
        fm = []
        dist = matcher.LM(torch.LongTensor([output]))[0]

        for w in cand:
            output_emb = matcher.embed(torch.LongTensor([output+[w]]))[0][-1] ## could be done in batch
            index, score = count_score(sourse, now, output_emb)
            index_list.append(index)
            fm.append(dist[matcher.get_cell_mates(w)].sum().item())

            score_list.append(score)

        score_list = torch.FloatTensor(score_list)
        qcm = F.softmax(score_list, -1).numpy()
        total_score = np.log(qcm) + 0.11*np.log(fm)
        k = np.argmax(total_score)

        output.append(cand[k])
        now = index_list[k]
        print(vocab_inv[output[-1]], now)
    
    output = [vocab_inv[i] for i in output]
    
    return output

In [151]:
def summary_batch(s, matcher):
    s_index = [[ vocab.get(w.lower(), vocab["<unk>"]) for w in sen] for sen in s]
    output_list = []
    for source in s_index:
        source = torch.LongTensor(source)
        s_emb = matcher.embed(torch.unsqueeze(source, 0))[0]
        vocab_list = matcher.candidate_list(source, 6).numpy()
        output = summary(s_emb, vocab_list, matcher)
        output_list.append(output)
    return output_list

In [152]:
test_input = []
test_ans = []
with open('/hdd/giga/train.article.txt') as f:
    for i in range(20):
        test_input.append(f.readline().strip())
with open('/hdd/giga/train.title.txt') as f:
    for i in range(len(test_input)):
        test_ans.append(f.readline().strip())
test_input = [list(nltk.word_tokenize(raw)) for raw in test_input]

In [153]:
result1 = summary_batch(test_input, matcher)
result2 = summary_batch(test_input, matcher_uni)

. 37
. 26
. 24
. 28
. 26
. 28
. 24
. 34
. 29
. 32
. 23
. 37
. 28
. 29
. 27
. 25
. 35
. 32
. 29
. 21
. 17
</S> 37
at 0
least 1
two 2
people 3
were 4
killed 5
in 6
a 7
attack 10
on 11
wednesday 21
. 26
australia 0
. 24
south 0
korea 1
's 2
nuclear 3
reactors 18
. 28
korea 1
on 2
monday 3
. 26
nicaragua 0
. 28
. 5
</S> 24
announced 10
second-half 22
losses 34
. 29
the 0
country 2
. 32
. 23
the 29
norwegians 33
were 34
said 35
. 37
the 17
analyst 26
said 27
. 28
. 29
. 27
korea 1
's 2
national 20
office 22
said 23
. 25
the 17
pro-russian 33
president 34
. 35
on 21
a 22
dark 23
. 32
philippine 0
share 1
prices 2
closed 3
altered 5
. 29
. 8
</S> 21


In [154]:
for i in range(len(test_input)):
    print('({})'.format(i))
    print('\t[ORI]:', ' '.join(test_input[i]))
    print('\t[BID]:', ' '.join(result1[i]))
    print('\t[UNI]:', ' '.join(result2[i]))
    print('\t[GRO]:', test_ans[i])

(0)
	[ORI]: australia 's current account deficit shrunk by a record # . # # billion dollars -lrb- # . # # billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed .
	[BID]: .
	[UNI]: . </S>
	[GRO]: australian current account deficit narrows sharply
(1)
	[ORI]: at least two people were killed in a suspected bomb attack on a passenger bus in the strife-torn southern philippines on monday , the military said .
	[BID]: .
	[UNI]: at least two people were killed in a attack on wednesday .
	[GRO]: at least two dead in southern philippines blast
(2)
	[ORI]: australian shares closed down # . # percent monday following a weak lead from the united states and lower commodity prices , dealers said .
	[BID]: .
	[UNI]: australia .
	[GRO]: australian stocks close down #.# percent
(3)
	[ORI]: south korea 's nuclear envoy kim sook urged north korea monday to restart work to disable its nuclear plants and stop its `` typical `` brinkmanship in negotiations .
	